# Eurostat

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

We opted for the Eurostat database to investigate overarching modal split trends in European transportation, focusing on car, train, and bus usage across all European countries since 1990. This choice was driven by the database's comprehensive coverage and longitudinal data, providing a solid foundation for gaining an initial understanding in the general transportation trends.

For all data sourced from the Eurostat database, the data cleaning was similar: Upon importing and reading the data, we filtered out aggregated entries and removed Turkey from our analysis to maintain a focus on European trends. To streamline our analysis, we eliminated flags indicating provisional or estimated data and treated missing values as NaN, ensuring a cleaner dataset.

Our primary inquiry centered on identifying and understanding the prevailing trends in Europe's modal split over the past decade, with a particular interest in any significant shifts or patterns. Our findings revealed a pronounced increase in car usage, a decline in bus usage, and a gradual resurgence in train usage, following a dip in the late '90s. Notably, the pandemic era marked a decline in public transportation use, contrasted by a significant surge in car usage. This trend solidified the car's position as the most favored mode of transportation, followed by buses and trains. This question was crucial as it guided our subsequent, more granular examination of the data.

In [ ]:
# Define the relative path to the CSV file
modal_split_file_path = '../data/eurostat/modal split percentage of inland passenger transport - tran_hv_psmod.csv'

# Read the CSV file
modal_split_data = pd.read_csv(modal_split_file_path)

# Convert to DataFrame
modal_split_df = pd.DataFrame(modal_split_data)

In [ ]:
# Split 'unit,vehicle,geo\\time' into separate columns for 'unit', 'vehicle', and 'country'
modal_split_df[['unit', 'vehicle', 'country']] = modal_split_df['unit,vehicle,geo\\time'].str.split(',', expand=True)
modal_split_df.drop('unit,vehicle,geo\\time', axis=1, inplace=True)

# Replace various forms of NaN representations
nan_values = {": c": np.nan, ":": np.nan, " :": np.nan, ": ": np.nan, ": p": np.nan}
modal_split_df.replace(nan_values, inplace=True)

# Exclude specific geographical entities
excluded_rows = ['EU27_2020', 'EU28', 'TRN_BUS_TOT_AVD', 'TR']
modal_split_df = modal_split_df[~modal_split_df['country'].isin(excluded_rows)]

# Function to clean non-numeric suffixes and handle empty strings
def clean_data_suffixes(df):
    for col in df.columns:
        # Remove non-numeric suffixes and convert empty strings to NaN
        df[col] = df[col].replace(to_replace=r'[^\d.]+', value='', regex=True).replace('', np.nan).astype(float)
    return df

# Separate tables for BUS_TOT, CAR, and TRN, set 'country' as the index, and drop 'vehicle' and 'unit' columns
bus_df = modal_split_df[modal_split_df['vehicle'] == 'BUS_TOT'].drop(['unit', 'vehicle'], axis=1).set_index('country')
car_df = modal_split_df[modal_split_df['vehicle'] == 'CAR'].drop(['unit', 'vehicle'], axis=1).set_index('country')
trn_df = modal_split_df[modal_split_df['vehicle'] == 'TRN'].drop(['unit', 'vehicle'], axis=1).set_index('country')

# Sort the columns (years) in ascending order for each DataFrame
bus_df = bus_df.reindex(sorted(bus_df.columns), axis=1)
car_df = car_df.reindex(sorted(car_df.columns), axis=1)
trn_df = trn_df.reindex(sorted(trn_df.columns), axis=1)

# Clean the 'BUS', 'CAR', and 'TRN' DataFrames
bus_df_cleaned = clean_data_suffixes(bus_df.copy())
car_df_cleaned = clean_data_suffixes(car_df.copy())
trn_df_cleaned = clean_data_suffixes(trn_df.copy())

# Display the first few rows of each cleaned DataFrame
print("Bus Data:")
display(bus_df_cleaned.head())
print("\nCar Data:")
display(car_df_cleaned.head())
print("\nTrain Data:")
display(trn_df_cleaned.head())

For visualizations, we used line plots to depict the temporal trends of bus, car, and train usage. These plots vividly illustrated how the popularity of each transportation mode evolved over time. Furthermore, made stacked bar plots for individual countries, offering a clearer visual breakdown of commuting preferences within each nation. 

In [ ]:
# Modified function to plot data with country code labels on each line, ensuring matching dimensions
def plot_data(df, title):
    plt.figure(figsize=(8, 4))

    # Convert the column names (years) to strings for the x-axis
    year_columns = df.columns.astype(str)

    # Loop through each country and plot its data with country codes on the lines
    for country in df.index:
        # Extract country data and align it with the year columns
        country_data = df.loc[country].reindex(year_columns).dropna()
        if not country_data.empty:
            plt.plot(year_columns[:len(country_data)], country_data)
            # Add text label near the last point of each line
            last_year = year_columns[len(country_data) - 1]
            last_value = country_data.iloc[-1]
            plt.text(last_year, last_value, country, fontsize=9)

    plt.title(title)
    plt.xticks(rotation=45)
    plt.ylabel('Percentage')
    plt.xlabel('Years')
    plt.tight_layout()
    plt.show()

# Plotting Bus, Car, and Train Data with country labels
plot_data(bus_df_cleaned, 'Percentage of Passenger Transport by Bus Over Years')
plot_data(car_df_cleaned, 'Percentage of Passenger Transport by Car Over Years')
plot_data(trn_df_cleaned, 'Percentage of Passenger Transport by Train Over Years')


In [ ]:
# Function to plot stacked bar plots for each country
def plot_stacked_bars_for_countries(bus_df, car_df, trn_df):
    # List of all countries
    countries = bus_df.index.unique()

    # Determine the grid size for subplots (fewer columns for more vertical spread)
    n_cols = 3  # Number of plots per row
    n_rows = -(-len(countries) // n_cols)  # Ceiling division for rows

    # Create figure and axes
    fig, axs = plt.subplots(n_rows, n_cols, figsize=(n_cols * 10, n_rows * 5), sharey=True)

    # Flatten the axes array for easy iteration
    axs = axs.flatten()

    # Loop through each country and create a stacked bar plot
    for i, country in enumerate(countries):
        # Extract data for the selected country
        bus_data = bus_df.loc[country]
        car_data = car_df.loc[country]
        trn_data = trn_df.loc[country]

        # Ensure all dataframes have the same set of columns (years)
        common_years = bus_data.index.intersection(car_data.index).intersection(trn_data.index)
        bus_data = bus_data.reindex(common_years)
        car_data = car_data.reindex(common_years)
        trn_data = trn_data.reindex(common_years)

        # Plotting each transportation mode
        axs[i].bar(common_years, bus_data, label='Bus', color='skyblue')
        axs[i].bar(common_years, car_data, bottom=bus_data, label='Car', color='lightgreen')
        axs[i].bar(common_years, trn_data, bottom=bus_data+car_data, label='Train', color='salmon')

        axs[i].set_title(country)
        axs[i].tick_params(axis='x', rotation=45)

    # Set the overall figure title and labels
    fig.suptitle('Modal Split of Transportation Over Years for Each Country', fontsize=16)
    fig.text(0.5, 0.04, 'Years', ha='center', va='center')
    fig.text(0.06, 0.5, 'Percentage', ha='center', va='center', rotation='vertical')

    # Hide any unused subplots
    for j in range(i+1, len(axs)):
        axs[j].axis('off')

    # Adjust layout and show the plot
    plt.tight_layout()
    plt.subplots_adjust(top=0.95)
    plt.show()

# Plotting stacked bar plots for each country
plot_stacked_bars_for_countries(bus_df_cleaned, car_df_cleaned, trn_df_cleaned)

In our analysis below, we examined the overall trends of passenger volume in relation to GDP. It was intriguing to investigate whether travel was influenced by GDP. However, we encountered a challenge in interpreting the graph, as it was quite perplexing. After we plotted the relative passenger volume to GDP over the years, the plot's significance remained unclear. Did it signify an improvement in public transportation efficiency? Was the decrease attributed to an increase in GDP, a decrease in the volume of travelers, or a reduction in the distance they traveled? Due to the lack of comprehensive documentation for the data sourced from the Eurostat database, we decided to omit this plot from our analysis.

In [ ]:
# Define the relative path to the CSV file
pkm_gdp_file_path = '../data/eurostat/volume of passenger transport relative to GDP(pkm divided by gdp) - tran_hv_pstra.csv'

# Read the CSV file
pkm_gdp_data = pd.read_csv(pkm_gdp_file_path)

# Convert to DataFrame
pkm_gdp_df = pd.DataFrame(pkm_gdp_data)

In [ ]:
# Extracting country codes by splitting the 'unit,geo\\time' column and selecting the second part
pkm_gdp_df['country'] = pkm_gdp_df['unit,geo\\time'].str.split(',').str[1]

# Exclude specific geographical entities
excluded_geos = ['EU27_2020', 'EU28', 'TR']
pkm_gdp_df = pkm_gdp_df[~pkm_gdp_df['country'].isin(excluded_geos)]

# Drop 'unit,geo\\time' column
pkm_gdp_df.drop('unit,geo\\time', axis=1, inplace=True)

# Set 'country' as the index
pkm_gdp_df.set_index('country', inplace=True)

# Replace ":" with NaN and remove ' p', ' e', or ' ep' from entries
pkm_gdp_df.replace({":": np.nan, " p": "", " e": "", " ep": ""}, regex=True, inplace=True)

# Convert the data to float
for col in pkm_gdp_df.columns:
    pkm_gdp_df[col] = pd.to_numeric(pkm_gdp_df[col], errors='coerce')

# Display the first few rows of the DataFrame
pkm_gdp_df.head()

In [ ]:
# Function to plot data with country code labels on each line
def plot_data_with_labels(df, title):
    plt.figure(figsize=(8, 4))

    # Convert the column names (years) to strings and sort them in ascending order for the x-axis
    year_columns = sorted(df.columns.astype(str))

    # Loop through each country and plot its data
    for country in df.index:
        # Ensure the data is in a float format for plotting and drop NaN values
        country_data = df.loc[country].reindex(year_columns).astype(float).dropna()
        if not country_data.empty:
            plt.plot(year_columns[:len(country_data)], country_data, label=country)
            # Add text label near the last point of each line
            last_year = year_columns[len(country_data) - 1]
            last_value = country_data.iloc[-1]
            plt.text(last_year, last_value, country, fontsize=9)

    plt.title(title)
    plt.xticks(rotation=45)
    plt.ylabel('pkm relative to GDP')
    plt.xlabel('Years')
    plt.tight_layout()
    plt.show()

# Plotting Passenger Transport Volume Relative to GDP (pkm divided by GDP) with country labels
plot_data_with_labels(pkm_gdp_df, 'Passenger Transport Volume Relative to GDP Over Years for Each Country')

We took a look at the volume of rail passengers over the years. The unit "pkm" represented passenger kilometers, which is the product of the distance traveled and the number of passengers on the journey. This allowed us to observe that, over time, the volume of passengers increased.

Similarly, with the modal split data sourced from Eurostat, after importing and reading the data, we applied filters to exclude aggregated entries and excluded Turkey from our analysis to maintain our focus on European trends. To streamline our analysis, we removed flags indicating provisional or estimated data and treated missing values as NaN, ensuring a cleaner dataset.

In [ ]:
# Define the relative path to the CSV file
rail_passengers_file_path = '../data/eurostat/number of rail passengers transported - rail_pa_total.csv'

# Read the CSV file
rail_passengers_data = pd.read_csv(rail_passengers_file_path)

# Convert to DataFrame
rail_passengers_df = pd.DataFrame(rail_passengers_data)

In [ ]:
# Split 'unit,geo\\time' into 'unit' and 'country'
rail_passengers_df[['unit', 'country']] = rail_passengers_df['unit,geo\\time'].str.split(',', expand=True)
rail_passengers_df.drop('unit,geo\\time', axis=1, inplace=True)

# Replace ": c", ":", " :", ": ", ": p", and any trailing non-numeric characters (like " p") from data columns only
for col in rail_passengers_df.columns.drop(['unit', 'country']):
    rail_passengers_df[col] = rail_passengers_df[col].replace({": c": np.nan, ":": np.nan, " :": np.nan, ": ": np.nan, ": p": np.nan}, inplace=False)
    rail_passengers_df[col] = rail_passengers_df[col].replace(to_replace=r'[^\d.]+', value='', regex=True)

# Convert the column names (representing years) to strings
rail_passengers_df.columns = rail_passengers_df.columns.astype(str)

# Filter out EU27_2007 and EU27_2020
rail_passengers_df = rail_passengers_df[~rail_passengers_df['country'].isin(['EU27_2007', 'EU27_2020', 'EU28', 'TR'])]

# Create the dataframe
df_mio_pkm = rail_passengers_df[rail_passengers_df['unit'] == 'MIO_PKM'].drop('unit', axis=1).set_index('country')

# Display the DataFrame
print("Million Passenger Kilometers (MIO_PKM):")
display(df_mio_pkm.head())


In [ ]:
plt.figure(figsize=(8, 4))

# Convert the column names (years) to strings
year_columns = df_mio_pkm.columns.astype(str)

# Loop through each country and plot its data with country codes on the lines
for country in df_mio_pkm.index:
    # Ensure the data is in a float format for plotting and drop NaN values
    country_data = df_mio_pkm.loc[country].astype(float).dropna()
    if not country_data.empty:
        plt.plot(year_columns[:len(country_data)], country_data)
        # Add text label near the last point of each line
        last_year = year_columns[len(country_data) - 1]
        last_value = country_data.iloc[-1]
        plt.text(last_year, last_value, country, fontsize=9)

plt.title('Million Passenger Kilometers (MIO_PKM) over Years')
plt.xticks(rotation=45)
plt.ylabel('Values')
plt.xlabel('Years')
plt.tight_layout()
plt.show()


Another interesting factor is the immigration levels of the EU countries over the year to see if more immigrants mean more public transportation use. However a strong correlation was not observed. Immigration's impact on public transportation usage varies due to factors such as individual preferences, socioeconomic status, and access to transportation options. The relationship is complex and may not exhibit a strong and consistent correlation across all contexts.

In [ ]:
# Define the relative path to the CSV file
immigration_file_path = '../data/eurostat/immigration.csv'

# Read the CSV file
immigration_data = pd.read_csv(immigration_file_path)

# Convert to DataFrame
immigration_df = pd.DataFrame(immigration_data)

In [ ]:
# Filter rows containing 'COMPLET'
immigration_df = immigration_df[immigration_df['freq,citizen,agedef,age,unit,sex,geo\\TIME_PERIOD'].str.contains('COMPLET')]

# Split the first column and drop the original column
immigration_df[['freq', 'citizen', 'agedef', 'age', 'unit', 'sex', 'country']] = immigration_df['freq,citizen,agedef,age,unit,sex,geo\\TIME_PERIOD'].str.split(',', expand=True)
immigration_df.drop('freq,citizen,agedef,age,unit,sex,geo\\TIME_PERIOD', axis=1, inplace=True)

# Clean the data in the year columns
for col in immigration_df.columns.drop(['freq', 'citizen', 'agedef', 'age', 'unit', 'sex', 'country']):
    immigration_df[col] = immigration_df[col].replace({": c": np.nan, ":": np.nan, " :": np.nan, ": ": np.nan, ": p": np.nan}, inplace=False)
    immigration_df[col] = immigration_df[col].replace(to_replace=r'[^\d.]+', value='', regex=True)

# Convert the column names (years) to strings
immigration_df.columns = immigration_df.columns.astype(str)

# Drop unnecessary columns
immigration_df_cleaned = immigration_df.drop(['freq', 'citizen', 'agedef', 'age', 'unit', 'sex'], axis=1)

# Set 'country' as the index of the DataFrame
immigration_df_final = immigration_df_cleaned.set_index('country')

# Drop the row with 'EU27_2020' in the 'country' index
immigration_df_final = immigration_df_final.drop(['EU27_2020', 'TR', 'EA19', 'EA20'], errors='ignore')

# Display the final DataFrame
immigration_df_final.head()

In [ ]:
# Function to plot data with country code labels on each line
def plot_data_with_labels(df, title):
    plt.figure(figsize=(8, 4))

    # Convert the column names (years) to strings
    year_columns = df.columns.astype(str)

    # Loop through each country and plot its data
    for country in df.index:
        # Ensure the data is in a float format for plotting and drop NaN values
        country_data = df.loc[country].astype(float).dropna()
        if not country_data.empty:
            plt.plot(year_columns[:len(country_data)], country_data, label=country)
            # Add text label near the last point of each line
            last_year = year_columns[len(country_data) - 1]
            last_value = country_data.iloc[-1]
            plt.text(last_year, last_value, country, fontsize=9)

    plt.title(title)
    plt.xticks(rotation=45)
    plt.ylabel('Number of Immigrants')
    plt.xlabel('Years')
    plt.tight_layout()
    plt.show()

# Plotting Immigration Data with country labels
plot_data_with_labels(immigration_df_final, 'Immigration Data Over Years for Each Country')


In [ ]:
# Define the relative path to the CSV file
hicp_file_path = '../data/eurostat/average-annual-HICP.csv'

# Read the CSV file
hicp_data = pd.read_csv(hicp_file_path)

# Convert to DataFrame
hicp_df = pd.DataFrame(hicp_data)

Finally, the Eurostat database contains information about the HICP for the European countries. The Harmonised Index of Consumer Prices (HICP) is a measure of inflation and price stability for the European Union (EU). It's designed to provide a consistent and comparable measure of consumer price inflation across EU member states. We want to see if inflation has an impact on the type of transportation that europeans use.

However, when we examined its relationship with transportation choices through a heatmap, an unexpected finding emerged: inflation appeared to have a positive correlation with car usage, contrary to the anticipated trend. Due to this counterintuitive result, we opted to exclude this plot from our analysis.

In [ ]:
# Split the first column and drop the original column
hicp_df[['freq', 'unit', 'coicop', 'country']] = hicp_df['freq,unit,coicop,geo\\TIME_PERIOD'].str.split(',', expand=True)
hicp_df.drop('freq,unit,coicop,geo\\TIME_PERIOD', axis=1, inplace=True)

# Before dropping rows, ensure 'country' column is of type string to avoid unexpected behavior
hicp_df['country'] = hicp_df['country'].astype(str)

# Drop the rows where 'country' column matches 'TR', 'EU27_2020', 'EA19', or 'EA20'
countries_to_drop = ['TR', 'EU27_2020', 'EA19', 'EA20']
hicp_df = hicp_df[~hicp_df['country'].isin(countries_to_drop)]

# Clean the data in the year columns
for col in hicp_df.columns.drop(['freq', 'unit', 'coicop', 'country']):
    hicp_df[col] = hicp_df[col].replace({":": np.nan, ": d": np.nan, " d": ""}, inplace=False, regex=True)
    hicp_df[col] = hicp_df[col].astype(float)

# Set 'country' as the index of the DataFrame
hicp_df_final = hicp_df.drop(['freq', 'unit', 'coicop'], axis=1).set_index('country')

# Display the cleaned DataFrame
hicp_df_final.head()

In [ ]:
# Function to plot data with country code labels on each line
def plot_data_with_labels(df, title):
    plt.figure(figsize=(8, 4))

    # Convert the column names (years) to strings
    year_columns = df.columns.astype(str)

    # Loop through each country and plot its data
    for country in df.index:
        # Ensure the data is in a float format for plotting and drop NaN values
        country_data = df.loc[country].dropna()
        if not country_data.empty:
            plt.plot(year_columns[:len(country_data)], country_data, label=country)
            # Add text label near the last point of each line
            last_year = year_columns[len(country_data) - 1]
            last_value = country_data.iloc[-1]
            plt.text(last_year, last_value, country, fontsize=9, ha='right')

    plt.title(title)
    plt.xticks(rotation=45)
    plt.ylabel('HICP Index')
    plt.xlabel('Years')
    plt.tight_layout()
    plt.show()

# Plotting HICP Data with country labels
plot_data_with_labels(hicp_df_final, 'HICP Data Over Years for Each Country')
